data for predicate detection

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import json
import cv2
import numpy as np
from easydict import EasyDict as edict
from google.colab import drive
drive.mount('/content/drive')

__C = edict()
cfg = __C
__C.DIR = '/content/drive/My Drive/VSimplE/'
__C.VRD_BATCH_NUM = 30


def generate_batch(N_total, N_each):
    num_batch = np.int32(N_total / N_each)
    if N_total % N_each == 0:
        index_box = range(N_total)
    else:
        index_box = np.empty(shape=[N_each * (num_batch + 1)], dtype=np.int32)
        index_box[0:N_total] = range(N_total)
        N_rest = N_each * (num_batch + 1) - N_total
        index_box[N_total:] = np.random.randint(0, N_total, N_rest)
    return index_box
###############################################################################


N_each_pred = cfg.VRD_BATCH_NUM
train_file_path = cfg.DIR + 'Dataset/VRD/json_dataset/annotations_train.json'
test_file_path = cfg.DIR + 'Dataset/VRD/json_dataset/annotations_test.json'
file_path = [train_file_path, test_file_path]
train_image_path = cfg.DIR + 'Dataset/VRD/sg_dataset/sg_train_images/'
test_image_path = cfg.DIR + 'Dataset/VRD/sg_dataset/sg_test_images/'
image_path = [train_image_path, test_image_path]
elmo_path = cfg.DIR + 'ELMo/Elmo.json'

with open(elmo_path, 'r') as b:
	elmo_list = json.load(b)

elmo_list = np.array(elmo_list, dtype=np.float32)
save_path  = cfg.DIR + 'input/vrd_roidb.npz'
save_path2 = cfg.DIR + 'input/co_occurency_tensor.npz'
save_path3 = cfg.DIR + 'input/P_s.npz'
save_path4 = cfg.DIR + 'input/P_o.npz'
save_path5 = cfg.DIR + 'input/P_sp.npz'
save_path6 = cfg.DIR + 'input/P_op.npz'
save_path7 = cfg.DIR + 'input/P_p.npz'


co_occurency_tensor = np.zeros((100,100,70))

for r in range(2):
	file_path_use = file_path[r]
	image_path_use = image_path[r]
	save_path_use = save_path[r]
	roidb = []
	with open(file_path_use,'r') as f:
		data=json.load(f)
		image_name = list(data.keys())
		len_img = len(image_name)
		t = 0

		for image_id in range(len_img):
			if (image_id+1)%100 == 0:
				print('image id is {0}'.format(image_id+1))

			roidb_temp = {}
			image_full_path = image_path_use + image_name[image_id]
			im = cv2.imread(image_full_path)
			if type(im) == type(None):
				continue

			im_shape = np.shape(im)
			im_h = im_shape[0]
			im_w = im_shape[1]

			roidb_temp['image'] = image_full_path
			roidb_temp['width'] = im_w
			roidb_temp['height'] = im_h

			d = data[image_name[image_id]]
			#print("d        :",d)
			# [
			# 	{'predicate': 0,
			# 	 'object': {'category': 97, 'bbox': [37, 296, 1, 520]},
			# 	 'subject': {'category': 50, 'bbox': [65, 97, 357, 411]}
			#   }
			# ]
			relation_length = len(d)
			if relation_length == 0:
				continue
			sb_new = np.zeros(shape=[relation_length,4])
			ob_new = np.zeros(shape=[relation_length,4])
			rela   = np.zeros(shape=[relation_length,])
			obj    = np.zeros(shape=[relation_length,])
			subj   = np.zeros(shape=[relation_length,])
			s = np.zeros(shape=[relation_length,])
			p = np.zeros(shape=[relation_length,])
			o = np.zeros(shape=[relation_length,])

			obj_elmo    = np.zeros(shape=[relation_length,128])
			subj_elmo   = np.zeros(shape=[relation_length,128])

			for relation_id in range(relation_length):
				relation = d[relation_id]

				obj[relation_id] = relation['object']['category']
				subj[relation_id] = relation['subject']['category']
				rela[relation_id] = relation['predicate']

				o = int(obj[relation_id])
				s = int(subj[relation_id])
				p = int(rela[relation_id])

				obj_elmo[relation_id] = elmo_list[o]
				subj_elmo[relation_id] = elmo_list[s]
				co_occurency_tensor[s][o][p] += 10.0

				ob_temp = relation['object']['bbox']
				sb_temp = relation['subject']['bbox']
				ob = [ob_temp[0],ob_temp[1],ob_temp[2],ob_temp[3]]
				sb = [sb_temp[0],sb_temp[1],sb_temp[2],sb_temp[3]]
				ob_new[relation_id][0:4] = [ob[2],ob[0],ob[3],ob[1]]
				sb_new[relation_id][0:4] = [sb[2],sb[0],sb[3],sb[1]]

			roidb_temp['sub_box_gt'] = sb_new[:] + 0.0
			roidb_temp['obj_box_gt'] = ob_new + 0.0
			roidb_temp['sub_gt'] = subj + 0.0
			roidb_temp['obj_gt'] = obj + 0.0
			roidb_temp['rela_gt'] = rela + 0.0
			roidb_temp['sub_elmo_gt'] = obj_elmo + 0.0
			roidb_temp['ob_elmo_gt'] = subj_elmo + 0.0
			roidb_temp['index_pred'] = generate_batch(len(rela), N_each_pred)
			roidb.append(roidb_temp)
		if r == 0:
			train_roidb = roidb
		elif r == 1:
			test_roidb = roidb
roidb = {}

roidb['train_roidb'] = train_roidb
roidb['test_roidb'] = test_roidb

#################### co_occurency_tensor normalization #################
sum_over_p= np.zeros((100,100))
for i in range(0,100):
	for j in range(0, 100):
		sum_p = 0.0
		for k in range(0,70):
			sum_p += co_occurency_tensor[i][j][k]
		sum_over_p[i][j] += sum_p
for i in range(0,100):
	for j in range(0, 100):
		sum_pp = sum_over_p[i][j] + 0.0
		if sum_pp == 0 :
			continue
		for k in range(0,70):
			co_occurency_tensor[i][j][k] /= sum_pp

######################	  P_sp & P_op & P_p	##############
P_op = np.sum(co_occurency_tensor , axis= 0)
P_sp = np.sum(co_occurency_tensor , axis= 1)
P_so = np.sum(co_occurency_tensor , axis= 2)

#################### normalization #####################
sum_over_o = np.zeros((100))
for i in range(0,100):
	sum_o = 0.0
	for j in range(0, 100):
		sum_o += P_so[i][j]
	sum_over_o[i] += sum_o

for i in range(0,100):
	sum_oo = sum_over_o[i] + 0.0
	if sum_oo == 0 :
		continue
	for j in range(0,100):
		P_so[i][j] /= sum_oo
#_________________________________
sum_over_p = np.zeros((100))
for i in range(0,100):
	sum_op = 0.0
	for j in range(0, 70):
		sum_op += P_op[i][j]
	sum_over_p[i] += sum_op

for i in range(0,100):
	sum_pp = sum_over_p[i] + 0.0
	if sum_pp == 0 :
		continue
	for j in range(0,70):
		P_op[i][j] /= sum_pp
#_________________________________
sum_over_s = np.zeros((100))
for i in range(0,100):
	sum_os = 0.0
	for j in range(0, 70):
		sum_os += P_so[i][j]
	sum_over_s[i] += sum_os

for i in range(0,100):
	sum_ss = sum_over_s[i] + 0.0
	if sum_ss == 0 :
		continue
	for j in range(0,70):
		P_so[i][j] /= sum_ss
###########################	  P_s & P_o	######################################
P_s = np.sum(P_so, axis = 0)
P_o = np.sum(P_so, axis = 1)
P_p = np.sum(P_sp, axis = 0)

###########################	 save	######################################
np.savez(save_path, roidb=roidb)
np.savez(save_path2,co_occurency_tensor)
np.savez(save_path3,P_s)
np.savez(save_path4,P_o)
np.savez(save_path5,P_sp)
np.savez(save_path6,P_op)
np.savez(save_path7,P_p)

print("Done!")